In [123]:
import sqlite3

def strToSqlSearch(theStr):
    return("%" + theStr + "%")

def graphVizHeader():
    return '''digraph G  {
    '''

def setTransform(theList):
    return(list(set(theList)))

searchTerm = "Influenza"

dbFile = "AMIDOL-SNOMED.sqlite3"
dbTable = "snomed"

conn = sqlite3.connect(dbFile)

curr = conn.cursor()

outf = open(searchTerm + 'map.dot', "w+")
outf.write(graphVizHeader())

leaves = setTransform([x[0] for x in curr.execute('SELECT id FROM snomed WHERE terms LIKE ?', [strToSqlSearch(searchTerm)]).fetchall()])
explored = []
toExplore = leaves

nodeColors = dict()
arcList = ""

print(len(leaves))

for leaf in leaves:
    annotations = curr.execute('SELECT annotations FROM snomed WHERE id = ?', [leaf]).fetchone()
    if (len(annotations[0]) > 0):
        nodeColors[leaf] = "blue"
    else:
        nodeColors[leaf] = "green"
        
iter = 0
        
while len(toExplore) > 0:
    iter += 1
    if (iter % 100 == 0):
        print(len(toExplore), len(explored), iter)
    nextState = toExplore.pop(0)
    explored.append(nextState)
    try:
        newLeaves = curr.execute('SELECT parents FROM snomed WHERE id = ?', [nextState]).fetchone()[0]
        newLeaves = setTransform([int(x) for x in newLeaves.split(',')])
        for leaf in newLeaves:
            annotations = curr.execute('SELECT annotations FROM snomed WHERE id = ?', [leaf]).fetchone()
            if (len(annotations[0]) > 0):
                nodeColors[leaf] = "red"
            arcList = arcList + str(nextState) + " -> " + str(leaf) + ";\n"
        explored.append(nextState)
        newLeaves = list(set(newLeaves) - set(explored))
        toExplore = setTransform(newLeaves + toExplore)
    except ValueError:
        pass
    
for key, value in nodeColors.iteritems():
    outf.write(str(key) + " [color=" + value + ", style=filled];\n")
    
outf.write(arcList)

conn.close()

outf.write('}\n')

outf.close()

750
(702, 190, 100)
(670, 381, 200)
(648, 578, 300)
(592, 768, 400)
(551, 968, 500)
(508, 1165, 600)
(435, 1365, 700)
(394, 1560, 800)
(354, 1752, 900)
(314, 1951, 1000)
(261, 2150, 1100)
(219, 2350, 1200)
(160, 2545, 1300)
(106, 2743, 1400)
(55, 2942, 1500)
